# 5. Searching KItems with the SDK

In this tutorial we see how to search existing Kitems

### 5.1. Setting up
Before you run this tutorial: make sure to have access to a DSMS-instance of your interest, alongwith with installation of this package and have establised access to the DSMS through DSMS-SDK (refer to [Connecting to DSMS](../dsms_sdk.md#connecting-to-dsms))


Now let us import the needed classes and functions for this tutorial.

In [1]:
from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [2]:
dsms = DSMS(env=".env")

### 5.2. Searching for KItems

In this section, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also want to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`- attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [3]:
item1 = KItem(
    name="Machine-1",
    ktype_id=dsms.ktypes.TestingMachine,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "TestingLab GmBH",
                       "Room Number": "A404",
                       "Description": "Bending Test Machine"
                       }
)

item2 = KItem(
    name="Machine-2",
    ktype_id=dsms.ktypes.TestingMachine,
    annotations=["https://w3id.org/steel/ProcessOntology/TestingMachine"],
    custom_properties={"Producer": "StressStrain GmBH",
                       "Room Number": "B500",
                       "Description": "Compression Test Machine"
                       }
)

item3 = KItem(
    name="Specimen-1", 
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item1],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties = {
        "Width": 0.5,
        "Length": [0.1, 0.2],
    }

)
item4 = KItem(
    name="Specimen-2",
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item2],
    annotations=["https://w3id.org/steel/ProcessOntology/TestPiece"],
    custom_properties = {
        "Width": 0.8,
        "Length": [0.8, 0.9],
    }
)

item5 = KItem(
    name="Research Institute ABC",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item1,item2],
    annotations=["www.researchBACiri.org/foo"],
)

dsms.commit()

/app/dsms/knowledge/kitem.py:573: UserWarning: A flat dictionary was provided for custom properties.
                    Will be transformed into `KItemCustomPropertiesModel`.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Room Number`. Cannot check if value is of correct type.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Description`. Cannot check if value is of correct type.
  warnings.warn(
/app/dsms/knowledge/webform.py:655: UserWarning: No webform was defined for entry `Length`. Cannot check if value is of correct type.
  warnings.warn(



<b> <p style="color:red;">Note : Here in this tutorial, we use dsms.search with `limit=1` to maintain readability but the user can adjust the variable `limit` as per requirement. </p>    </b> 


Now, we are apply to search for e.g. kitems of type `TestingMachine`:

In [4]:
result = dsms.search(ktypes=[dsms.ktypes.Specimen], limit=2)
print(result)
print("Name of the first kitem:")
print(result.hits[0].kitem.name)

hits:
- kitem:
    name: Specimen-1
    id: b0505a0f-8739-4162-9cad-240139338cd4
    ktype_id: specimen
    slug: specimen-1-b0505a0f
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestPiece
      label: TestPiece
      namespace: https://w3id.org/steel/ProcessOntology
    attachments:
    - name: subgraph.ttl
      content: null
    linked_kitems:
    - id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
      name: Machine-1
      slug: machine-1-01bb6156
      ktype_id: testing-machine
      summary: null
      avatar_exists: false
      annotations:
      - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
        label: TestingMachine
        namespace: https://w3id.org/steel/ProcessOntology
      linked_kitems:
      - id: a0688216-78fe-41da-9d55-0b897cece9db
      - id: b0505a0f-8739-4162-9cad-240139338cd4
      external_links: []
      contacts: []
      authors:
      - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
      linked_affiliations: []
      attachme

... and for all of type `Organization` and `Testingmachine`:

In [5]:
for result in dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.TestingMachine], limit=1):
    print(result.kitem)
    print("fuzziness: ", result.fuzzy)
    print("\n")
    

kitem:
  name: Machine-1
  id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
  ktype_id: testing-machine
  slug: machine-1-01bb6156
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: a0688216-78fe-41da-9d55-0b897cece9db
    name: Research Institute ABC
    slug: researchinstituteabc-a0688216
    ktype_id: organization
    summary: null
    avatar_exists: false
    annotations:
    - iri: www.researchBACiri.org/foo
      label: foo
      namespace: www.researchBACiri.org
    linked_kitems:
    - id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
    - id: 74438e78-0311-46a6-86dc-204f3ba8e550
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments: []
    user_groups: []
    custom_properties: null
    created_at: '2025-01-28T12:0

... or for all of type `Specimen` with `Specimen1` in the name:

In [6]:
for result in dsms.search(query="Specimen-1", ktypes=[dsms.ktypes.Specimen], limit=1):
    print(result)

kitem:
  name: Specimen-1
  id: b0505a0f-8739-4162-9cad-240139338cd4
  ktype_id: specimen
  slug: specimen-1-b0505a0f
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestPiece
    label: TestPiece
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
    name: Machine-1
    slug: machine-1-01bb6156
    ktype_id: testing-machine
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems:
    - id: a0688216-78fe-41da-9d55-0b897cece9db
    - id: b0505a0f-8739-4162-9cad-240139338cd4
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments:
    - name: subgraph.ttl
      content: null
    user_groups: []


... and for all of type `Organization` with the annotation `www.researchBACiri.org/foo`:

In [7]:
for result in dsms.search(
          ktypes=[dsms.ktypes.Organization], annotations=["www.researchBACiri.org/foo"], limit=1
    ):
    print(result)


kitem:
  name: Research Institute ABC
  id: a0688216-78fe-41da-9d55-0b897cece9db
  ktype_id: organization
  slug: researchinstituteabc-a0688216
  annotations:
  - iri: www.researchBACiri.org/foo
    label: foo
    namespace: www.researchBACiri.org
  attachments: []
  linked_kitems:
  - id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
    name: Machine-1
    slug: machine-1-01bb6156
    ktype_id: testing-machine
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems:
    - id: a0688216-78fe-41da-9d55-0b897cece9db
    - id: b0505a0f-8739-4162-9cad-240139338cd4
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments:
    - name: subgraph.ttl
      content: null
    user_groups: []
    custom_properties:
      sections:
      - id: id717

## 5.3. Fetching linked KItems from a KItem

In the beginning under **5.1** we created some kitems and linked each other. Now we want to fetch the linked kitems and display them to the user. For this we use the `linked_kitems` attribute.

In [8]:
item5.linked_kitems

[linked_kitem:
   id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
   name: Machine-1
   slug: machine-1-01bb6156
   ktype_id: testing-machine
   summary: null
   avatar_exists: false
   annotations:
   - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
     label: TestingMachine
     namespace: https://w3id.org/steel/ProcessOntology
   linked_kitems:
   - id: a0688216-78fe-41da-9d55-0b897cece9db
   - id: b0505a0f-8739-4162-9cad-240139338cd4
   external_links: []
   contacts: []
   authors:
   - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
   linked_affiliations: []
   attachments:
   - name: subgraph.ttl
     content: null
   user_groups: []
   custom_properties:
     sections:
     - id: id717506ec647db
       name: ''
       entries:
       - id: id699fb85c65df4
         type: Text
         label: Producer
         value: TestingLab GmBH
         measurementUnit: null
         relationMapping: null
     - id: id1738065716294lzgvq9
       name: Misc
       entries:
       - id: id

However, this linked KItem is not a "real" KItem. Due to performance reasons, we only display a slim representation of the linked KItem. Imagine if the linked KItem also has a linked KItem... this might lead to an infinite recursion. We can fetch the "real" KItem by using the `fetch` method of the linked KItem:


In [9]:
item5.linked_kitems[0].fetch()

kitem:
  name: Machine-1
  id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
  ktype_id: testing-machine
  slug: machine-1-01bb6156
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: a0688216-78fe-41da-9d55-0b897cece9db
    name: Research Institute ABC
    slug: researchinstituteabc-a0688216
    ktype_id: organization
    summary: null
    avatar_exists: false
    annotations:
    - iri: www.researchBACiri.org/foo
      label: foo
      namespace: www.researchBACiri.org
    linked_kitems:
    - id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
    - id: 74438e78-0311-46a6-86dc-204f3ba8e550
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments: []
    user_groups: []
    custom_properties: null
    created_at: '2025-01-28T12:0

We can also get the linked KItems grouped by annotation:

In [10]:
item5.linked_kitems.by_annotation

{'https://w3id.org/steel/ProcessOntology/TestingMachine': [linked_kitem:
    id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
    name: Machine-1
    slug: machine-1-01bb6156
    ktype_id: testing-machine
    summary: null
    avatar_exists: false
    annotations:
    - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
      label: TestingMachine
      namespace: https://w3id.org/steel/ProcessOntology
    linked_kitems:
    - id: a0688216-78fe-41da-9d55-0b897cece9db
    - id: b0505a0f-8739-4162-9cad-240139338cd4
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments:
    - name: subgraph.ttl
      content: null
    user_groups: []
    custom_properties:
      sections:
      - id: id717506ec647db
        name: ''
        entries:
        - id: id699fb85c65df4
          type: Text
          label: Producer
          value: TestingLab GmBH
          measurementUnit: null
          relationMappi

Again, we are able to fetch the "real" KItem by using the `fetch` method of the linked KItem:

In [11]:
item5.linked_kitems.by_annotation["https://w3id.org/steel/ProcessOntology/TestingMachine"][0].fetch()

kitem:
  name: Machine-1
  id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
  ktype_id: testing-machine
  slug: machine-1-01bb6156
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: a0688216-78fe-41da-9d55-0b897cece9db
    name: Research Institute ABC
    slug: researchinstituteabc-a0688216
    ktype_id: organization
    summary: null
    avatar_exists: false
    annotations:
    - iri: www.researchBACiri.org/foo
      label: foo
      namespace: www.researchBACiri.org
    linked_kitems:
    - id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
    - id: 74438e78-0311-46a6-86dc-204f3ba8e550
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments: []
    user_groups: []
    custom_properties: null
    created_at: '2025-01-28T12:0

Additionally, we can retrieve the linked KItems grouped by ktype:

In [12]:
item5.linked_kitems.by_ktype

{ktype:
   id: testing-machine
   name: Testing Machine
   webform:
     semantics_enabled: true
     sections_enabled: false
     class_mapping:
     - https://w3id.org/steel/ProcessOntology/TestingMachine
     sections:
     - id: id717506ec647db
       name: ''
       inputs:
       - id: id699fb85c65df4
         label: Producer
         widget: Text
         required: false
         value: null
         hint: null
         hidden: false
         ignore: false
         select_options: []
         measurement_unit: null
         relation_mapping:
           iri: https://w3id.org/steel/ProcessOntology/hasIdentifier
           type: data_property
           class_iri: null
         relation_mapping_extra: null
         multiple_selection: false
         knowledge_type: null
         range_options: null
         placeholder: null
       hidden: false
   json_schema:
     iri:
     - https://w3id.org/steel/ProcessOntology/TestingMachine
     suffix: slug
     source: testing-machine[*]
 

... and retrieve them by type:

In [13]:
item5.linked_kitems.by_ktype[dsms.ktypes.TestingMachine][0].fetch()

kitem:
  name: Machine-1
  id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
  ktype_id: testing-machine
  slug: machine-1-01bb6156
  annotations:
  - iri: https://w3id.org/steel/ProcessOntology/TestingMachine
    label: TestingMachine
    namespace: https://w3id.org/steel/ProcessOntology
  attachments:
  - name: subgraph.ttl
    content: null
  linked_kitems:
  - id: a0688216-78fe-41da-9d55-0b897cece9db
    name: Research Institute ABC
    slug: researchinstituteabc-a0688216
    ktype_id: organization
    summary: null
    avatar_exists: false
    annotations:
    - iri: www.researchBACiri.org/foo
      label: foo
      namespace: www.researchBACiri.org
    linked_kitems:
    - id: 01bb6156-dc37-4bed-86f2-cfd22ad2ffcc
    - id: 74438e78-0311-46a6-86dc-204f3ba8e550
    external_links: []
    contacts: []
    authors:
    - user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
    linked_affiliations: []
    attachments: []
    user_groups: []
    custom_properties: null
    created_at: '2025-01-28T12:0

Clean up the DSMS from the tutortial:

In [14]:
del dsms[item1]
del dsms[item2]
del dsms[item3]
del dsms[item4]
del dsms[item5]

dsms.commit()